In [10]:
import sys
import json
import csv
import numpy as np
from pandas.io.json import json_normalize

In [11]:
businessjson = 'dataset/business.json'
checkinjson = 'dataset/checkin.json'

# Load Business data
business_json_lines = [json.loads( l.strip() ) for l in open(businessjson).readlines() ]
df = json_normalize(business_json_lines)


In [16]:
# Get records from Yelp set for only restaurants
df = df[df['categories'].astype(str).str.lower().str.contains('restaurant') == True]
df = df.reset_index(drop=True)

#Remove Columns of least significance
df = df.drop(['address','attributes.AcceptsInsurance','attributes.AgesAllowed','attributes.BYOB','attributes.ByAppointmentOnly','attributes.Corkage','attributes.DietaryRestrictions.dairy-free','attributes.DietaryRestrictions.gluten-free','attributes.DietaryRestrictions.halal','attributes.DietaryRestrictions.kosher','attributes.DietaryRestrictions.soy-free','attributes.DietaryRestrictions.vegan','attributes.DietaryRestrictions.vegetarian','attributes.HairSpecializesIn.africanamerican','attributes.HairSpecializesIn.asian','attributes.HairSpecializesIn.coloring','attributes.HairSpecializesIn.curly','attributes.HairSpecializesIn.extensions','attributes.HairSpecializesIn.kids','attributes.HairSpecializesIn.perms','attributes.HairSpecializesIn.straightperms','attributes.Open24Hours','attributes.RestaurantsCounterService','business_id','categories','hours.Friday','hours.Monday','hours.Saturday','hours.Sunday','hours.Thursday','hours.Tuesday','hours.Wednesday','latitude','longitude','name','neighborhood','state'],axis=1)

# Classify the records
df['target'] = [1 if df['stars'][x] >= 3.5 and df['review_count'][x] >= 30 else 0 for x in range(len(df))]
df.to_csv('business_csv_filter.csv', encoding='utf-8', index=False)
print df['target'].value_counts()

0    37140
1    14485
Name: target, dtype: int64


In [33]:
# Seggregting data to positive and negative buckets.
pos_bucket= [df.loc[i]  for i in range(len(df)) if df['target'][i] == 1]
neg_negative = [df.loc[i]  for i in range(len(df)) if df['target'][i] == 0]
print len(pos_bucket)
print len(neg_negative)


14485
37140


full_bar
